<a href="https://colab.research.google.com/github/abhishekmishra-bareilly/Deep-Learning/blob/main/Data_augmentation_cat_vs_dog(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

## Import the dataset from kaggel

In [14]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [15]:
!kaggle datasets download -d salader/dogs-vs-cats

100% 1.06G/1.06G [00:51<00:00, 24.3MB/s]
100% 1.06G/1.06G [00:51<00:00, 22.4MB/s]


In [16]:
# The file is in zip file to unzio
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [17]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels='inferred',
    label_mode = 'int',

    # Batch size
    batch_size=32,

    # For reshaing the images
    image_size=(256,256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels='inferred',
    label_mode = 'int',

    # Batch size
    batch_size=32,

    # For reshaing the images
    image_size=(256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


## Normalize the data

In [18]:
# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label


# Applying the function
train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

## Creating CNN Model

In [19]:
model = Sequential()
    
model.add(Conv2D(32, kernel_size=(3,3),padding='valid',activation ='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size =(2,2),strides = 2,padding='valid'))

model.add(Conv2D(64, kernel_size=(3,3),padding='valid',activation ='relu'))
model.add(MaxPooling2D(pool_size =(2,2),strides = 2,padding='valid'))

model.add(Conv2D(128, kernel_size=(3,3),padding='valid',activation ='relu'))
model.add(MaxPooling2D(pool_size =(2,2),strides = 2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))



In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 128)      0

In [21]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [22]:
# Fitting the model
hisstory =  model.fit(train_ds,epochs=10,validation_data=validation_ds)

Epoch 1/10
625/625 [==============================] - 69s 94ms/step - loss: 0.6620 - accuracy: 0.6119 - val_loss: 0.6016 - val_accuracy: 0.6528
Epoch 2/10
625/625 [==============================] - 57s 91ms/step - loss: 0.4982 - accuracy: 0.7567 - val_loss: 0.4825 - val_accuracy: 0.7614
Epoch 3/10
625/625 [==============================] - 63s 100ms/step - loss: 0.3673 - accuracy: 0.8330 - val_loss: 0.5479 - val_accuracy: 0.7654
Epoch 4/10
625/625 [==============================] - 59s 94ms/step - loss: 0.2129 - accuracy: 0.9127 - val_loss: 0.9082 - val_accuracy: 0.7576
Epoch 5/10
625/625 [==============================] - 59s 94ms/step - loss: 0.1112 - accuracy: 0.9578 - val_loss: 0.9695 - val_accuracy: 0.7610
Epoch 6/10
625/625 [==============================] - 57s 91ms/step - loss: 0.0739 - accuracy: 0.9747 - val_loss: 1.1893 - val_accuracy: 0.7684
Epoch 7/10
625/625 [==============================] - 57s 91ms/step - loss: 0.0609 - accuracy: 0.9798 - val_loss: 1.2218 - val_accuracy

## Data augmentation

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
# Adding rescale, rotation_range, width_shift_range, height_shift_range,
# shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# Note that the validation data should not be augmented!
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/train',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 32 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        '/content/test',
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [25]:
model = Sequential()
    
model.add(Conv2D(32, kernel_size=(3,3),padding='valid',activation ='relu',input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size =(2,2),strides = 2,padding='valid'))

model.add(Conv2D(64, kernel_size=(3,3),padding='valid',activation ='relu'))
model.add(MaxPooling2D(pool_size =(2,2),strides = 2,padding='valid'))

model.add(Conv2D(128, kernel_size=(3,3),padding='valid',activation ='relu'))
model.add(MaxPooling2D(pool_size =(2,2),strides = 2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))



In [26]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [35]:
# Fitting the model
hisstory =  model.fit(train_generator,batch_size=64,epochs=2,validation_data=validation_generator)

Epoch 1/2


InvalidArgumentError: ignored